In [1]:
import pdfplumber
import re
from PyPDF2 import PdfReader
from tqdm import tqdm

In [2]:
%pip install pycryptodome

Note: you may need to restart the kernel to use updated packages.


In [3]:
def remove_first_line(text):
    return text[text.find('\n') + 1:]

In [4]:
def extract_text_pdfplumber(pdf_path):
    full_text = []

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            
            if not text:
                continue

            text = remove_first_line(text) # верхний колонтитул

            # text = re.sub(r"\bГЛАВА\s?\d+\b", "", text, flags=re.IGNORECASE)

            text = re.sub(r"Рис.\s?\d+.\d+.", "", text, flags=re.IGNORECASE) # подписи к рисункам
            
            # text = re.sub(r"^\d+$", "", text, flags=re.MULTILINE)

            full_text.append(text)
    
    return "\n\n".join(full_text)

In [5]:
def extract_text_pypdf2(pdf_path):
    full_text = []

    reader = PdfReader(pdf_path)

    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        
        text = page.extract_text()

        if not text:
            continue

        text = remove_first_line(text)

        text = re.sub(r"Рис.\s?\d+.\d+.", "", text, flags=re.IGNORECASE)  
        
        full_text.append(text)

   
    return "\n\n".join(full_text)

In [6]:
def extract_text_pypdf2_ind_pages(pdf_path, output_dir = "output", pdf_name = "pdf"):

    reader = PdfReader(pdf_path)

    for page_num in tqdm(range(len(reader.pages))):
        page = reader.pages[page_num]
        
        text = page.extract_text()

        if not text:
            continue

        text = remove_first_line(text)

        text = re.sub(r"Рис.\s?\d+.\d+.", "", text, flags=re.IGNORECASE)  
        
        with open(f"{output_dir}/{pdf_name}_page_{page_num}.txt",
                  "w", encoding="utf-8") as f:
            f.write(text)

   
    print(f"Done parsing. Results in {output_dir}")

In [7]:
small_pdf_path = "data/LakeIV_B-100-105.pdf"
big_pdf_path = "orig_data/LakeIV_B.pdf"

In [8]:
extracted_text_pdfplumber = extract_text_pdfplumber(small_pdf_path)

extracted_text_pypdf2 = extract_text_pypdf2(small_pdf_path)

# print(extracted_text_pypdf2[:5000])



In [9]:
pdf_path = big_pdf_path

In [11]:
extract_text_pypdf2_ind_pages(pdf_path=pdf_path, output_dir="full_file" )

100%|██████████| 1232/1232 [00:48<00:00, 25.46it/s]

Done parsing. Results in full_file


In [84]:
pdf_name = pdf_path.split(".")[0]

with open(f"outputs/{pdf_name}_pypdf2.txt", "w", encoding="utf-8") as f:
    f.write(extracted_text_pypdf2)

with open(f"outputs/{pdf_name}_pdfplumber.txt", "w", encoding="utf-8") as f:
    f.write(extracted_text_pdfplumber)